In [1]:
import os 
import pandas as pd

import itertools as it

import warnings
warnings.filterwarnings('ignore')

# Making pics info DataFrame

In [ ]:
path = 'dataset/lfw_funneled'

pairs = ''
with open(f'{path}/pairs.txt', 'r') as file:
    pairs = file.read()

df = pd.DataFrame(list(map(lambda x: x.split('\t'), pairs.split('\n')[1:301])), columns=['name', 'pic1', 'pic2'])
df = (pd.melt(df, id_vars='name', value_vars=['pic1', 'pic2'], var_name='picNum', value_name='pic')
        .drop('picNum', axis=1)
        .sort_values(by=['name', 'pic'])
        .drop_duplicates()
        .reset_index(drop=True))

df['pic'] = df['pic'].apply(lambda x: x.zfill(4))
df['path_x1'] = str(path) + '/' + df['name'] + '/' + df['name'] + '_' + df['pic'] + '.jpg'
df.head()

In [ ]:
pics_to_scale_path = df['path_x1'].values
pics_to_scale = list(map(lambda x: x.split('/')[-1][:-4]+ '_result.jpg', pics_to_scale_path))
print(f'There are {len(pics_to_scale)} unique pics to be scaled')

# Scaling and adding path info to pics DataFrame

In [ ]:
# Тут надо самому ставить нужный скейл: 2 3 или 4. Так как памяти нехватает + настроить сохранение
# в нужную папку сложна (не хочется заморачиваться), то надо перезапускать и руками переименовывать папку.
scaled_pics = os.listdir('dataset/output')
for pic, pic_path in zip(pics_to_scale, pics_to_scale_path):
    if pic in scaled_pics:
        continue
    abc = f'--file={pic_path} --scale=4'
    %run sr.py {abc}

In [ ]:
for scale in [2, 3, 4]:
    df[f'path_x{scale}'] = f'dataset/output_x{scale}/' + df[f'path_x1'].apply(lambda x: x.split('/')[-1][:-4]) + '_result.jpg'
df.head()

In [ ]:
df.to_csv('pics_df.csv')

# Making DataFrame for classification

#### Получили всевозможные пары фоточек для каждого человека

In [ ]:
classification_data = []
for name, name_df in df[['name', 'pic']].groupby('name'):
    pics = name_df['pic'].values
    pics_comb = list(it.combinations(pics, 2))
    pics_name_comb = list(map(lambda x: [name]+list(x), pics_comb))
    for comb in pics_name_comb:
        classification_data.append(comb)
classification_df = pd.DataFrame(classification_data, columns=['name', 'pic1', 'pic2'])
classification_df.head()

In [ ]:
for pic_num in [1, 2]:
    classification_df = (pd.merge(classification_df, df, 
                                  left_on=['name', f'pic{pic_num}'], right_on=['name', 'pic'], 
                                  how='left', suffixes=('_1', '_2')))
classification_df.drop(['pic_1', 'pic_2'], axis=1, inplace=True)
classification_df.head()

In [ ]:
classification_df.to_csv('classification_pics_df.csv')

# Classification

In [2]:
from recognizer import Recognizer
from skimage.io import imread

THRESHOLD = 0.9

In [3]:
data = pd.read_csv('classification_pics_df.csv', index_col=0)
data.head()

,name,pic1,pic2,path_x1_1,path_x2_1,path_x3_1,path_x4_1,path_x1_2,path_x2_2,path_x3_2,path_x4_2
0,Abel_Pacheco,1,4,dataset/lfw_funneled/Abel_Pacheco/Abel_Pacheco...,dataset/output_x2/Abel_Pacheco_0001_result.jpg,dataset/output_x3/Abel_Pacheco_0001_result.jpg,dataset/output_x4/Abel_Pacheco_0001_result.jpg,dataset/lfw_funneled/Abel_Pacheco/Abel_Pacheco...,dataset/output_x2/Abel_Pacheco_0004_result.jpg,dataset/output_x3/Abel_Pacheco_0004_result.jpg,dataset/output_x4/Abel_Pacheco_0004_result.jpg
1,Akhmed_Zakayev,1,2,dataset/lfw_funneled/Akhmed_Zakayev/Akhmed_Zak...,dataset/output_x2/Akhmed_Zakayev_0001_result.jpg,dataset/output_x3/Akhmed_Zakayev_0001_result.jpg,dataset/output_x4/Akhmed_Zakayev_0001_result.jpg,dataset/lfw_funneled/Akhmed_Zakayev/Akhmed_Zak...,dataset/output_x2/Akhmed_Zakayev_0002_result.jpg,dataset/output_x3/Akhmed_Zakayev_0002_result.jpg,dataset/output_x4/Akhmed_Zakayev_0002_result.jpg
2,Akhmed_Zakayev,1,3,dataset/lfw_funneled/Akhmed_Zakayev/Akhmed_Zak...,dataset/output_x2/Akhmed_Zakayev_0001_result.jpg,dataset/output_x3/Akhmed_Zakayev_0001_result.jpg,dataset/output_x4/Akhmed_Zakayev_0001_result.jpg,dataset/lfw_funneled/Akhmed_Zakayev/Akhmed_Zak...,dataset/output_x2/Akhmed_Zakayev_0003_result.jpg,dataset/output_x3/Akhmed_Zakayev_0003_result.jpg,dataset/output_x4/Akhmed_Zakayev_0003_result.jpg
3,Akhmed_Zakayev,2,3,dataset/lfw_funneled/Akhmed_Zakayev/Akhmed_Zak...,dataset/output_x2/Akhmed_Zakayev_0002_result.jpg,dataset/output_x3/Akhmed_Zakayev_0002_result.jpg,dataset/output_x4/Akhmed_Zakayev_0002_result.jpg,dataset/lfw_funneled/Akhmed_Zakayev/Akhmed_Zak...,dataset/output_x2/Akhmed_Zakayev_0003_result.jpg,dataset/output_x3/Akhmed_Zakayev_0003_result.jpg,dataset/output_x4/Akhmed_Zakayev_0003_result.jpg
4,Amber_Tamblyn,1,2,dataset/lfw_funneled/Amber_Tamblyn/Amber_Tambl...,dataset/output_x2/Amber_Tamblyn_0001_result.jpg,dataset/output_x3/Amber_Tamblyn_0001_result.jpg,dataset/output_x4/Amber_Tamblyn_0001_result.jpg,dataset/lfw_funneled/Amber_Tamblyn/Amber_Tambl...,dataset/output_x2/Amber_Tamblyn_0002_result.jpg,dataset/output_x3/Amber_Tamblyn_0002_result.jpg,dataset/output_x4/Amber_Tamblyn_0002_result.jpg


In [4]:
recognizer = Recognizer()

def recongize(row):
    photo1 = imread(row.ix[0])
    photo2 = imread(row.ix[1])
    similarity = recognizer.get_best_similarity(photo1, photo2)
    return similarity < THRESHOLD

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Restoring parameters from ./facenet_models/facenet/model-20180408-102900.ckpt-90


In [6]:
%%time
for scale in [1, 2, 3, 4]:
    data[f'rec_x{scale}'] = data[[f'path_x{scale}_1', f'path_x{scale}_2']].apply(recongize, axis=1)
    print(f'Scale x{scale} is done.')

Scale x1 is done.
Scale x2 is done.
Scale x3 is done.
Scale x4 is done.
Wall time: 56min 29s


In [38]:
classification_results = pd.DataFrame(['x1', 'x2', 'x3', 'x4'], columns=['scale'])
classification_results['accuracy'] = (data[['rec_x1', 'rec_x2', 'rec_x3', 'rec_x4']]
                                      .apply(lambda col: round(col.value_counts(normalize=True) * 100, 2))
                                      .iloc[0].values)
classification_results

,scale,accuracy
0,x1,80.94
1,x2,83.21
2,x3,83.93
3,x4,83.81


In [39]:
classification_results.to_csv('classification_results_DCSCN.csv')